In [ ]:
import torch
import math
import preprocess
import perturbation
from torch.nn import MSELoss
import torch.nn.functional as F
from tqdm.auto import tqdm

In [ ]:
# 设置任务和模型
task_name = "SST-2" # ["CoLA", "SST-2", "MRPC", "STS-B", "QQP", "MNLI-m", "MNLI-mm", "QNLI", "RTE", "WNLI"]
model_name = "models/bert-base-uncased" # [bert-base-uncased, roberta-base]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# 设置超参数
# 训练参数
SEED = 42
BATCH_SIZE = 32
EPOCH = 5
# 对抗训练参数
adv_init_epsilon = 1e-2 # 初始化扰动
adv_init_type = "zero" # ["zero","rand","randn"]
sampling_times_theta = 10 # theta采样次数
sampling_times_delta = 3 # delta采样次数
sampling_noise_theta = 0 # 采样噪声
sampling_noise_delta = 0 # 采样噪声
sampling_step_theta = 3e-5 # theta采样步长
sampling_step_delta = 1e-2 # theta采样步长
lambda_s = 1 # lambda λ
beta = 0.1 # beta β

In [ ]:
model, dataloader, metric= preprocess.preprocess(task_name, model_name, BATCH_SIZE, SEED)
train_dataloader,eval_dataloader,test_dataloader = dataloader

In [ ]:
def ls(P, Q):
    task_type = "classification" if task_name != "STS-B" else "regression"
    if(task_type == "classification"):
        return F.kl_div(P.softmax(dim=-1).log(), Q.softmax(dim=-1), reduction='batchmean') + F.kl_div(Q.softmax(dim=-1).log(), P.softmax(dim=-1), reduction='batchmean')
    elif(task_type == "regression"):
        return MSELoss(P, Q, reduction="sum")

In [ ]:
def SGLD(z, grad, size, step, epsilon):
    noise = perturbation.init_delta(size, epsilon=epsilon, init_type="randn")
    z = z - step * grad + math.sqrt(2 * step) * noise
    return z

In [ ]:
import time
train_start = time.time()
file = open("run: "+ str(int(train_start)) +".log","w")

In [ ]:
# Training
print(time.ctime(), file=file)
print("*"*20, "Training", "*"*20, file=file)  # 训练任务
print("TASK:", task_name, file=file)
print("MODEL:", model_name, file=file)
print("DEVICE:", device, file=file)
print("="*16, "General Training", "="*16, file=file)  # 常规训练参数
print("EPOCH_NUM:", EPOCH, file=file)
print("BATCH_SIZE:", BATCH_SIZE, file=file)
print("="*18, "MAT Training", "="*18, file=file)  # MAT训练参数
print("Adversarial_Training_type:", "MAT", file=file)
print("Adversarial_init_epsilon:", adv_init_epsilon, file=file)
print("Adversarial_init_type:", adv_init_type, file=file)
print("Sampling_times_theta:", sampling_times_theta, file=file)
print("Sampling_times_delta:", sampling_times_delta, file=file)
print("Sampling_noise_theta:", sampling_noise_theta, file=file)
print("Sampling_noise_delta:", sampling_noise_delta, file=file)
print("Sampling_step_theta:", sampling_step_theta, file=file)
print("Sampling_step_delta:", sampling_step_delta, file=file)
print("lambda:", lambda_s, file=file)
print("beta:", beta, file=file)
print("*"*50, file=file)
model.to(device)
progress_bar = tqdm(range(EPOCH * len(train_dataloader)))
eval_metric_list = []
for i in range(EPOCH):
    print("-"*20, "EPOCH:", i, "-"*20, file=file)
    print("Training...", end='', file=file)
    model.train()

    for batch in train_dataloader:
        for t in batch:
            batch[t] = batch[t].to(device)

        # [begin] MAT Training
        # 1.init delta & inputs
        ## 1.1 获得batch的word_embedding
        if "bert-" in model_name:
            word_embedding = model.bert.embeddings.word_embeddings(batch["input_ids"])
        elif "roberta-" in model_name:
            word_embedding = model.roberta.embeddings.word_embeddings(batch["input_ids"])
        
        ## 1.2 初始化[抽样扰动delta_k]和[分布扰动mean_delta]
        delta = perturbation.init_delta(word_embedding.size(), epsilon=adv_init_epsilon, init_type=adv_init_type)
        delta.requires_grad = True
        mean_delta = delta  # 初始化delta的分布均值mean_delta

        ## 1.3 初始化模型输入inputs
        if "bert-" in model_name:  # bert模型输入inputs: "attention_mask","labels","token_type_ids",「inputs_embeds」和「input_ids」参数二选一
            inputs = {"attention_mask": batch["attention_mask"],"labels": batch["labels"], "token_type_ids": batch["token_type_ids"]}
        elif "roberta-" in model_name:  # roberta模型输入inputs: "attention_mask","labels",「inputs_embeds」和「input_ids」参数二选一
            inputs = {"attention_mask": batch["attention_mask"], "labels": batch["labels"]}

        ## 1.4 备份模型参数
        back_parameters = model.state_dict()
        mean_theta = model.state_dict()

        # 2.stochastic gradient langevin dynamics sampling
        ## 2.1 sampling perturbation (delta)
        for k in range(sampling_times_delta):
            ### 构造带有扰动的输入
            inputs["inputs_embeds"] = delta + word_embedding.detach()
            ### 前向传播
            loss_adv = ls(model(**inputs).logits, model(**batch).logits)
            ### 反向传播
            loss_adv.backward()
            ### SGLD采样
            delta.data = SGLD(delta.data, - delta.grad.data, delta.size(), sampling_step_delta, sampling_noise_delta)
            delta.requires_grad = True
            ### 更新扰动的分布均值
            mean_delta = beta * mean_delta + (1 - beta) * delta

        ## 2.2 sampling model parameters (theta)
        for k in range(sampling_times_theta):
            ### 清空模型参数的梯度
            for p in model.parameters():
                p.grad.zero_()
            ### 构造带有扰动的输入
            inputs["inputs_embeds"] = mean_delta.detach() + word_embedding.detach()
            ### 前向传播
            loss_sum = model(**batch).loss + lambda_s * ls(model(**inputs).logits, model(**batch).logits)
            ### 反向传播
            loss_sum.backward()
            ### SGLD采样
            for p in model.parameters():
                p.data = SGLD(p.data, p.grad.data, p.size(), sampling_step_theta, sampling_noise_theta)
            ### 更新模型参数的分布均值
            new_model_param = model.state_dict()
            for name in mean_theta:
                mean_theta[name] = beta * mean_theta[name] + (1 - beta) * new_model_param[name]

        # 3.update model parameters
        for name in back_parameters:
            back_parameters[name] = beta * back_parameters[name] + (1 - beta) * mean_theta[name]
        model.load_state_dict(back_parameters) # 更新这次迭代的模型参数
        # [end] MAT Training
        progress_bar.update(1)

    print("\rEvaling...", end='', file=file)
    model.eval()
    for batch in eval_dataloader:
        for t in batch:
            batch[t] = batch[t].to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1) if task_name != "STS-B" else outputs.logits.squeeze()
        metric.add_batch(predictions=predictions, references=batch["labels"])
    score = metric.compute()
    eval_metric_list.append(score)
    print("\rMetric:", score, file=file)
    print("-"*50, file=file)

# Best score in eval
score_list = []
for m in eval_metric_list:
    score_list.append(list(m.values())[0])
print("*"*19, "Best Score", "*"*19, file=file)
print("EPOCH:", score_list.index(max(score_list)), file=file)
print("Metric:", eval_metric_list[score_list.index(max(score_list))], file=file)
print("*"*50, file=file)


In [ ]:
file.close()